In [49]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

In [50]:
df = pd.read_csv('ff_race_50.csv')
df.head()

,Place,First,Last,City,State,Age,Division,Time,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,NaN,NaN,NaN
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,NaN,NaN,NaN
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,NaN,NaN,NaN
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,NaN,NaN,NaN
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,NaN,NaN,NaN


In [51]:
df2 = df.dropna(axis=1)
df2.head()

,Place,First,Last,City,State,Age,Division,Time
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17


In [52]:
df2['fulname'] = df2['First'] + ' ' + df2['Last']
df2.head()

C:\Users\keerthi\AppData\Local\Temp\ipykernel_22112\1243878517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fulname'] = df2['First'] + ' ' + df2['Last']


,Place,First,Last,City,State,Age,Division,Time,fulname
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,Daniel Wilson
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,Eric Davis
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,Stewart Edwards
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,Ron Hammett
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,Seth Cain


In [53]:
df2['Time'] = pd.to_timedelta(df2['Time'])

C:\Users\keerthi\AppData\Local\Temp\ipykernel_22112\2600047365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Time'] = pd.to_timedelta(df2['Time'])


In [54]:
df2['Total_minutes'] = (df2['Time'].dt.total_seconds() / 60).round().astype(int)
df2.head()

C:\Users\keerthi\AppData\Local\Temp\ipykernel_22112\3319008970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Total_minutes'] = (df2['Time'].dt.total_seconds() / 60).round().astype(int)


,Place,First,Last,City,State,Age,Division,Time,fulname,Total_minutes
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582


In [55]:
df2.rename(columns = {'Divison' : 'gender'}, inplace = True)

C:\Users\keerthi\AppData\Local\Temp\ipykernel_22112\3847095021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns = {'Divison' : 'gender'}, inplace = True)


In [57]:
def get_lat_long(city,state):
    address = f"{city},{state}"
    try :
        geolocator = Nominatim(user_agent="running",timeout=10)
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except geocoderTimedOut:
        return None, None

In [58]:
df2['lat'], df2['long'] = zip(*df2.apply(lambda row: get_lat_long(row['City'], row['State']), axis=1))

In [59]:
df2.head()

,Place,First,Last,City,State,Age,Division,Time,fulname,Total_minutes,lat,long
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538,39.613699,-86.109543
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565,29.025813,-80.927127
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565,28.600277,-81.673964
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.737294,-81.113421


In [60]:
df2['latlong'] = df2['lat'].astype(str) + ',' + df2['long'].astype(str)

In [61]:
df2.head()

,Place,First,Last,City,State,Age,Division,Time,fulname,Total_minutes,lat,long,latlong
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752,"36.1563122,-95.9927516"
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538,39.613699,-86.109543,"39.6136987,-86.1095429"
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565,29.025813,-80.927127,"29.0258132,-80.9271271"
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565,28.600277,-81.673964,"28.6002769,-81.673964"
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.737294,-81.113421,"28.737294,-81.1134205"


# Firefall Race Data Analysis

## Overview
This notebook analyzes data from a 50-mile ultramarathon race, processing the raw data to create a clean, analysis-ready dataset. The race features runners from various states, age groups, and skill levels competing in a challenging endurance event.

## Data Processing Steps

1. **Data Import and Initial Exploration**
   - Load raw race results from CSV file
   - Examine data structure and available fields
   - Identify cleaning needs and missing values

2. **Data Cleaning and Transformation**
   - Remove columns with missing values
   - Create full name from first and last names
   - Convert race times to minutes for easier analysis
   - Standardize column names (e.g., 'Division' to 'gender')
   - Calculate total race time in minutes for quantitative analysis

3. **Geocoding**
   - Add latitude and longitude based on city and state using GeoPy
   - Implement rate limiting and error handling for API requests
   - Create combined location coordinates for mapping visualizations

4. **Data Export**
   - Save the cleaned dataset for further analysis in SQL or other tools

## Analysis Potential
The processed dataset enables several types of analysis:
- Geographic distribution of participants
- Performance differences by gender and age groups
- Correlation between location (altitude, climate) and performance
- Identification of competitive regions for ultramarathon running